In [13]:
function count_amidakuji(n, m)
    # あみだくじの本数が2以下では計算できない
    @assert n > 2

    # 3次元配列を初期化 (n-1 × n × n)
    # BigInt型を使用して大きな数値に対応
    a = zeros(BigInt, n-1, n, n)
    
    # 初期状態: 最初の層の対角要素を1に設定
    for j in 1:n
        a[1, j, j] = BigInt(1)
    end
    
    # m回の変換を行う
    for loop in 1:m
        # 次の状態を保持する配列
        next = zeros(BigInt, n-1, n, n)
        
        # 累積フェーズ
        for j in 1:n
            for k in 1:n
                # 最初の層
                next[1, j, k] = a[1, j, k] + a[2, j, k]
                
                # 中間層
                for i in 2:(n-2)
                    next[i, j, k] = next[i-1, j, k] + a[i+1, j, k]
                end
                
                # 最後の層
                if n > 3  # n-3が有効な場合のみ
                    next[n-1, j, k] = next[n-2, j, k]
                end
            end
        end
        
        # 交換フェーズ: 各層で隣接要素を交換
        for i in 1:(n-1)
            for j in 1:n
                next[i, j, i], next[i, j, i+1] = next[i, j, i+1], next[i, j, i]
            end
        end
        
        # aを次の状態で更新
        a = next
    end
    
    # 結果を計算して出力
    total = 0
    
    for j in 1:n
        row_values = []
        for k in 1:n
            # 各位置(j,k)について全層の合計を計算
            sum_val = sum(a[i, j, k] for i in 1:(n-1))
            push!(row_values, sum_val)
            
            # 最後の列の合計を総計に追加
            if k == n
                total += sum_val
            end
        end
        println(join(row_values, " "))
    end
    
    println("total: ", total)
    return total
end

# メイン処理
function main()
    println("あみだくじの本数(n)と横線の本数(m)を入力してください:")
    n, m = parse.(Int, split(readline()))
    
    # 大きな計算になることを警告
    if n > 20 || m > 50
        println("注意: 大きな計算になります。しばらく時間がかかる場合があります。")
    end
    
    count_amidakuji(n, m)
end

# プログラム実行
main()

あみだくじの本数(n)と横線の本数(m)を入力してください:


stdin>  4 3


11 6 3 1
6 6 6 3
3 6 6 6
1 3 6 11
total: 21


21

In [15]:
using Random

"""
    generate_random_amidakuji(n, m)

n本の縦線とm本の横線を持つランダムなあみだくじを生成します。
横線は重複しない位置にランダムに配置されます。

返り値:
- horizontal_lines: 横線の位置を表す配列。各要素は (row, col) の形式で、
  rowは上から何段目か、colは左から何本目の縦線の位置を示します。
  横線は col と col+1 の間に配置されます。
"""
function generate_random_amidakuji(n, m)
    # 横線の本数がm本を超えないようにする
    max_possible_lines = (n - 1) * (n - 1)
    actual_m = min(m, max_possible_lines)
    
    # 可能な横線の位置をすべて列挙
    possible_positions = [(row, col) for row in 1:(n-1) for col in 1:(n-1)]
    
    # ランダムにm本選ぶ
    shuffle!(possible_positions)
    horizontal_lines = possible_positions[1:actual_m]
    
    # 横線の位置を上から下、左から右の順にソート
    sort!(horizontal_lines)
    
    return horizontal_lines
end

"""
    trace_path(horizontal_lines, n, start_pos)

あみだくじの経路をトレースし、スタート位置からの到着位置を返します。
"""
function trace_path(horizontal_lines, n, start_pos)
    current_pos = start_pos
    
    # 上から下へ各行を処理
    for row in 1:(n-1)
        # 現在位置の右側に横線があるか
        if (row, current_pos) in horizontal_lines
            current_pos += 1
        # 現在位置の左側に横線があるか
        elseif current_pos > 1 && (row, current_pos-1) in horizontal_lines
            current_pos -= 1
        end
    end
    
    return current_pos
end

"""
    display_amidakuji(horizontal_lines, n, paths=nothing)

あみだくじを視覚的に表示します。
paths が指定されている場合は、特定の経路を強調表示します。
"""
function display_amidakuji(horizontal_lines, n, paths=nothing)
    # 表示用の文字列をセットアップ
    rows = []
    
    # 上部の数字（出発点）
    top_numbers = join([lpad(i, 3) for i in 1:n])
    push!(rows, "  " * top_numbers)
    
    # 縦線と横線を描画
    for row in 1:(n-1)
        # 縦線を描画
        vertical_line = ""
        for col in 1:n
            vertical_line *= " | "
        end
        push!(rows, vertical_line)
        
        # 横線を描画
        horizontal_line = " "
        for col in 1:(n-1)
            if (row, col) in horizontal_lines
                horizontal_line *= "|---"
            else
                horizontal_line *= "    "
            end
        end
        horizontal_line *= "|"
        push!(rows, horizontal_line)
    end
    
    # 最後の縦線
    vertical_line = ""
    for col in 1:n
        vertical_line *= " | "
    end
    push!(rows, vertical_line)
    
    # 下部の数字（到着点）
    bottom_numbers = join([lpad(i, 3) for i in 1:n])
    push!(rows, "  " * bottom_numbers)
    
    # 経路が指定されている場合は、強調表示
    if paths !== nothing
        # 各出発点からの経路を表示
        path_info = ["出発点 $start → 到着点 $(paths[start])" for start in 1:n]
        push!(rows, "")
        push!(rows, "経路情報:")
        for info in path_info
            push!(rows, info)
        end
    end
    
    # 結果を出力
    for row in rows
        println(row)
    end
end

"""
    save_amidakuji_to_file(horizontal_lines, n, paths, filename)

あみだくじの情報をファイルに保存します。
"""
function save_amidakuji_to_file(horizontal_lines, n, paths, filename)
    open(filename, "w") do file
        write(file, "n=$n\n")
        write(file, "横線の数=$(length(horizontal_lines))\n")
        write(file, "横線の位置:\n")
        for (row, col) in horizontal_lines
            write(file, "($row, $col)\n")
        end
        write(file, "\n経路情報:\n")
        for start in 1:n
            write(file, "出発点 $start → 到着点 $(paths[start])\n")
        end
    end
    println("あみだくじ情報を $filename に保存しました。")
end

"""
    generate_multiple_amidakuji(n, m, count)

n本の縦線とm本の横線を持つあみだくじをcount個生成します。
"""
function generate_multiple_amidakuji(n, m, count)
    result = []
    for i in 1:count
        horizontal_lines = generate_random_amidakuji(n, m)
        paths = [trace_path(horizontal_lines, n, start) for start in 1:n]
        push!(result, (horizontal_lines, paths))
    end
    return result
end

# メイン関数
function main()
    println("あみだくじランダム生成プログラム")
    println("----------------------------------------")
    
    # パラメータ入力
    print("縦線の本数(n)を入力してください: ")
    n = parse(Int, readline())
    
    print("横線の最大本数(m)を入力してください: ")
    m = parse(Int, readline())
    
    print("生成するあみだくじの数を入力してください: ")
    count = parse(Int, readline())
    
    # 複数のあみだくじを生成
    println("\n$count 個のあみだくじを生成します...")
    amidakujis = generate_multiple_amidakuji(n, m, count)
    
    # 各あみだくじを表示
    for (i, (horizontal_lines, paths)) in enumerate(amidakujis)
        println("\n========== あみだくじ #$i ==========")
        display_amidakuji(horizontal_lines, n, paths)
        
        # 保存するか尋ねる
        print("\nこのあみだくじを保存しますか？ (y/n): ")
        if lowercase(readline()) == "y"
            filename = "amidakuji_$(n)_$(length(horizontal_lines))_$i.txt"
            save_amidakuji_to_file(horizontal_lines, n, paths, filename)
        end
    end
    
    println("\n処理が完了しました。")
end

# プログラム実行
main()

あみだくじランダム生成プログラム
----------------------------------------
縦線の本数(n)を入力してください: 

stdin>  30


横線の最大本数(m)を入力してください: 

stdin>  60


生成するあみだくじの数を入力してください: 

...


========== あみだくじ #4 ==========
    1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                     |---    |---|---|---                |---                |---                                |---|
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                                                                                     |---                            |
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
         |---                                                    |---                                                |
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                         |---                                                                                        |
 |  |  |  |  |  |  |  |  |  |  |  |

stdin>  n



========== あみだくじ #5 ==========
    1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                             |---                                        |---                                        |
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                             |---                                                        |---                        |
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                                                                             |---    |---                            |
 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | 
                 |---    |---                                                                                        |
 |  |  |  |  |  |  |  |  |  |  |  |

stdin>  n



処理が完了しました。
